In [138]:
# gpu number setting
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1' ## gpu 번호 셋팅 윤건 :0, 기준 : 1, 준형 :2,

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.3.1
keras version :  2.4.0
GPU available ? :  True


In [139]:
import pandas as pd
import pickle

In [140]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [141]:
with open('../uplus_exp/raw_mod_2.pickle', 'rb') as f:
    data = pickle.load(f)

* buy_history_price \\N 인곳 0원으로 채우기

In [142]:
list(data[data['buy_history_price'] == '\\N'].index)

[1,
 2,
 3,
 6,
 7,
 8,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 55,
 56,
 57,
 58,
 59,
 61,
 64,
 69,
 74,
 75,
 76,
 77,
 79,
 81,
 82,
 83,
 87,
 89,
 90,
 91,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 173,
 174,
 175,
 177,
 178,
 181,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 193,
 194,
 195,
 201,
 206,
 207,
 212,
 213,
 214,
 215,
 216,
 219,
 220,
 222,
 223,
 224,
 225,
 226,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 236,
 237,
 238,
 239

In [143]:
data.loc[list(data[data['buy_history_price'] == '\\N'].index), ['buy_history_price']] = 0

In [43]:
sparse_features = ['sa_id', 'album_id', 'buy_seg'] + [str(i) for i in range(32)]
dense_features = ['release_date', 'run_time', 'movie_meta_price', 'i30_meta_price', 'buy_tot',
                  'amt_1_4', 'amt_2_4', 'amt_3_4', 'amt_4_4']
ambiguous_features = [ 'agree_yn', 'meta_genre', 'genre_large', 'genre_mid', 'genre_small']
unnecessary_features = ['category_id', # NaN 약 650만 중에 25만 정도.
                        's_time', 'e_time', # future
                        'watch_duration', # future
                        'buy_history_price', # \\N 값이 약 482만 개
                        'vod_s_point', 'vod_e_point', 
                        'album_name',
                        'view_no',
                        'fod', 'buy_1_2', 'buy_3',
                        'amt_r_gabun',
                        'watch_ratio',
                        'weekdays', 'weekends', 'dawn', 'morning', 'afternoon', 'evening',
                        'current_rate']
target = ['ncf_rating']

In [44]:
data['sa_id'].unique()

array([205101491713, 205112221633, 205112403631, ..., 'M17022300005',
       'M17062200019', 'M18061900001'], dtype=object)

In [45]:
print(len(data['album_id'].unique()))
print(len(data['sa_id'].unique()))

10570
968100


* all data -> user_table & content_table

In [46]:
user_tab = data[['sa_id', 'buy_seg', 'buy_tot']]
cont_tab = data[['album_id', 'release_date', 'run_time',
                 'movie_meta_price', 'i30_meta_price', 'buy_history_price',
                'amt_1_4', 'amt_2_4', 'amt_3_4', 'amt_4_4'] + [str(i) for i in range(32)]]

In [47]:
cont_tab

,album_id,release_date,run_time,movie_meta_price,i30_meta_price,buy_history_price,amt_1_4,amt_2_4,amt_3_4,amt_4_4,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,M01198J812PPV00,20190424.0,181,2500.0,2500,2500,1,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,M011449172PPV00,20020322.0,115,1400.0,1400,0,1,0,0,0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,M0114CB383PPV00,20031002.0,124,0.0,0,0,1,0,0,0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,M0111CMB08PPV00,20040730.0,100,0.0,1400,0,1,0,0,0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,M0119A8236PPV00,20190911.0,115,2500.0,2500,2500,0,1,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6759188,M011849006PPV00,20180307.0,101,1400.0,1400,1400,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6759189,M01203Q384PPV00,20200122.0,102,9900.0,9900,0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False
6759190,M01206H267PPV00,20200604.0,102,5000.0,5000,10000,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6759191,M01206U088PPV00,20200617.0,102,9900.0,9900,18900,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [48]:
user_tab

,sa_id,buy_seg,buy_tot
0,205101491713,VIEW_NO,0
1,205112221633,VIEW_NO,0
2,205112221633,VIEW_NO,0
3,205112221633,VIEW_NO,0
4,205112403631,BUY_1_2,1
...,...,...,...
6759188,M18061900001,BUY_3,14
6759189,M18061900001,BUY_3,14
6759190,M18061900001,BUY_3,14
6759191,M18061900001,BUY_3,14


* duplicate rows 제거

In [54]:
user_uni_tab = user_tab.loc[list(user_tab['sa_id'].drop_duplicates().index)]
cont_uni_tab = cont_tab.loc[list(cont_tab['album_id'].drop_duplicates().index)]

* randomly join?

In [145]:
import random
import numpy as np

In [74]:
def rand_sample(x, n):
    return x.loc[random.sample(list(x.index), n)]

In [164]:
len_u = len(user_uni_tab)
len_c = len(cont_uni_tab)

In [200]:
random.randint(0, len_u)

268853

* 시간 줄이기...

In [206]:
tmp = time.time()
user_uni_tab.iloc[[random.randint(0, len_u)]]
print(time.time() - tmp)

0.0008425712585449219


In [205]:
tmp = time.time()
rand_sample(user_uni_tab, 1)
print(time.time() - tmp)

0.11789846420288086


In [379]:
user_uni_tab.sample(1)

,sa_id,buy_seg,buy_tot
2362920,500032262343,BUY_1_2,1


In [417]:
tmp_t = time.time()
u = user_uni_tab.iloc[[random.randint(0, len_u-1)]]
c = cont_uni_tab.iloc[[random.randint(0, len_c-1)]]
print(time.time()-tmp_t)

0.0014066696166992188


In [406]:
tmp_t = time.time()
u = user_uni_tab.sample(1)
c = cont_uni_tab.sample(1)
print(time.time()-tmp_t)

0.020831584930419922


In [418]:
import tqdm

In [421]:
u

,sa_id,buy_seg,buy_tot
5694080,500174964788,FOD,0


In [422]:
c

,album_id,release_date,run_time,movie_meta_price,i30_meta_price,buy_history_price,amt_1_4,amt_2_4,amt_3_4,amt_4_4,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
454732,M01148E407PPV00,20140821.0,109,1400.0,900,1400,1,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False


In [423]:
tmp

,sa_id,buy_seg,buy_tot,album_id,release_date,run_time,movie_meta_price,i30_meta_price,buy_history_price,amt_1_4,amt_2_4,amt_3_4,amt_4_4,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,500174964788,FOD,0,M01148E407PPV00,20140821.0,109,1400.0,900,1400,1.0,0.0,0.0,0.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False


In [424]:
res = None

for i in tqdm.tqdm(range(7000000)):
#     u = rand_sample(user_uni_tab, 1)
#     c = rand_sample(cont_uni_tab, 1)
    t1 = time.time()
    u = user_uni_tab.iloc[[random.randint(0, len_u-1)]]
    c = cont_uni_tab.iloc[[random.randint(0, len_c-1)]]
    

    t2 = time.time()
    values = u.values.tolist()[0] + c.values.tolist()[0]
    #values = u.values.tolist()[0] + c.values.tolist()[0]
    
    t3 = time.time()
    cols = list(u.columns) + list(c.columns)

    t4 = time.time()
    tmp = pd.DataFrame(data=np.reshape(values, newshape=[1, -1]), columns = cols)

    t5 = time.time()
    res = pd.concat([res, tmp])
    t6 = time.time()
    
    #break
    
print(t2-t1)
print(t3-t2)
print(t4-t3)
print(t5-t4)
print(t6-t5)



  0%|          | 7964/7000000 [00:13<3:18:13, 587.86it/s]


  0%|          | 15877/7000000 [00:26<3:20:48, 579.65it/s]


  0%|          | 23603/7000000 [00:40<3:22:33, 574.03it/s]


  0%|          | 31322/7000000 [00:53<3:24:36, 567.62it/s]


  1%|          | 39030/7000000 [01:07<3:18:32, 584.33it/s]


...

 19%|█▉        | 1355973/7000000 [38:42<2:40:57, 584.41it/s]


 19%|█▉        | 1363628/7000000 [38:55<2:39:39, 588.39it/s]


 20%|█▉        | 1371295/7000000 [39:08<2:38:53, 590.40it/s]


 20%|█▉        | 1375894/7000000 [39:16<2:47:11, 560.66it/s]

KeyboardInterrupt: 

In [ ]:
res

* 기존 테이블에 있던 조합은 제거